In [2]:
import fiona
import rasterio
from rasterio.tools.mask import mask
from rasterio.warp import reproject, RESAMPLING, transform, transform_bounds
from utils import check_create_folder
pathimg='/Users/yamaoka/landsat/downloads/'
pathshp='/Users/yamaoka/landuse/shapefiles/'
#bounds = [-97.938385,30.516863,-97.561485,30.098665]
#shpfile = 'austin_tx.shp'

In [3]:
def toepsg32614(inputshpfile,outputshpfile):
    tmp = gpd.GeoDataFrame.from_file(inputshpfile)
    tmpepsg32614 = tmp.to_crs({'init':'epsg:32614'})
    tmpepsg32614.to_file(outputshpfile)

In [4]:
def cropbybb(infile,outfile,left,bottom,right,top):
    with rasterio.open(infile) as src:

        bounds = transform_bounds(
                                {
                                'proj': 'longlat',
                                'ellps': 'WGS84',
                                'datum': 'WGS84',
                                'no_defs': True
                                },
                                src.crs,
                                top=top,bottom=bottom,right=right,left=left
                                )

        window = src.window(*bounds)

        out_kwargs = src.meta.copy()
        out_kwargs.update({
                        'driver': 'GTiff',
                        'height': window[0][1] - window[0][0],
                        'width': window[1][1] - window[1][0],
                        'transform': src.window_transform(window)
                        })
        with rasterio.open(outfile, 'w', **out_kwargs) as out:
            out.write(src.read(window=window))


In [5]:
def cropbyshp(infile,outfile,shpfile) :    

    with fiona.open(shpfile, "r") as shapefile:     
     geoms = [feature["geometry"] for feature in shapefile]

    with rasterio.open(infile) as src:
     out_image, out_transform = mask(src, geoms, crop=True)
     out_meta = src.meta.copy()
  
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(outfile, "w", **out_meta) as dest:
     dest.write(out_image)


In [6]:
myshpfile= pathshp+'austin_tx_epsg32614.shp'

In [19]:
#root  = "LC80270392013179LGN00" #AUSTIN 2013-06-28
#root  = "LC80270392017030LGN00" #AUSTIN 2017-01-30
root  = "LC80270392014118LGN00" #AUSTIN 2014-04-28
inpath = pathimg+root+'/'
outpath = inpath+'clipped/'
check_create_folder(outpath)

'/Users/yamaoka/landsat/downloads/LC80270392014118LGN00/clipped/'

In [20]:
clfile = root+"_B1.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B2.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B3.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B4.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B5.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B6.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)

In [21]:
clfile = root+"_B7.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B8.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B9.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_B10.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)
clfile = root+"_BQA.TIF"
cropbyshp(inpath+clfile,outpath+clfile,myshpfile)

In [23]:
clfile = root+"_B8.TIF"
cropbyshp(inpath+clfile,"test_r.tif","tempepsg3214.shp")